In [1]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import HYPERPARAMETERS

In [2]:
params = HYPERPARAMETERS
params['batch_size'] = 8
params['verbose'] = True

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')
    
print(params)

{'batch_size': 8, 'checkpoints_dir': '/home/sergevkim/git/FacesClassification/checkpoints', 'checkpoint_filename': '', 'disable_cuda': False, 'imgs_dir': '/home/sergevkim/git/FacesClassification/data/CelebaHQ', 'label': 'Male', 'labels_filename': '/home/sergevkim/git/FacesClassification/data/list_attr_celeba.txt', 'logs_dir': '/home/sergevkim/git/FacesClassification/logs', 'n_epochs': 10, 'n_imgs': 30000, 'verbose': True, 'version': '0.1', 'device': device(type='cuda', index=0)}


In [3]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

0 0.5
50 0.5
100 0.375
150 0.375
200 0.625
250 0.25


KeyboardInterrupt: 